In [43]:
print("womp womp")

womp womp
time: 0 ns (started: 2023-12-13 20:03:55 -05:00)


In [44]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.linear_model import LinearRegression

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2023-12-13 20:03:55 -05:00)


In [45]:
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Heart Attack Risk Predicition\heart_attack_prediction_dataset.csv"

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,Male,208,158,72,0,0,1,0,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,21,Male,389,165,98,1,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,21,Female,324,174,72,1,0,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,84,Male,383,163,73,1,1,1,0,1,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,66,Male,318,91,93,1,1,1,1,0,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


time: 15 ms (started: 2023-12-13 20:03:55 -05:00)


In [46]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})

time: 0 ns (started: 2023-12-13 20:03:55 -05:00)


In [47]:
heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
unique = heart_attack['Hemisphere'].unique()
print(unique)
unique_country = heart_attack['Country'].unique()
print(unique_country)
unique_diet = heart_attack['Diet'].unique()
print(unique_diet)
unique_continent = heart_attack['Continent'].unique()
print(unique_continent)

heart_attack.head()

[1 0]
[-1  1]
[-1  1]
[ 0 -1  1]
[4 3 2 0 1 5]


,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,1,208,158,72,0,0,1,0,0,...,6.615001,261404,31.251233,286,0,6,-1,4,-1,0
1,21,1,389,165,98,1,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,1,3,1,0
2,21,0,324,174,72,1,0,0,0,0,...,9.463426,235282,28.176571,587,4,4,1,2,1,0
3,84,1,383,163,73,1,1,1,0,1,...,7.648981,125640,36.464704,378,3,4,1,3,1,0
4,66,1,318,91,93,1,1,1,1,0,...,1.514821,160555,21.809144,231,1,5,-1,0,1,0


time: 16 ms (started: 2023-12-13 20:03:55 -05:00)


In [48]:
# Extract 4 shared features between all datasets and target
X = heart_attack[['Sex', 'Age', 'Cholesterol', 'Heart Attack Risk', 'Blood Pressure']].values

y_diab = heart_attack[['Diabetes']].values.reshape(-1, 1)
y_obesity = heart_attack[['Obesity']].values.reshape(-1, 1)
y_alcCon = heart_attack[['Alcohol Consumption']].values.reshape(-1, 1)
y_ExHrsPrWk = heart_attack[['Exercise Hours Per Week']].values.reshape(-1, 1)
y_diet = heart_attack[['Diet']].values.reshape(-1, 1)
y_previousHeartProb = heart_attack[['Previous Heart Problems']].values.reshape(-1, 1)
y_continent = heart_attack[['Continent']].values.reshape(-1, 1)
y_hr = heart_attack['Heart Rate'].values.reshape(-1, 1)
y_famhist = heart_attack['Family History'].values.reshape(-1, 1)
y_smoking = heart_attack['Smoking'].values.reshape(-1, 1)
y_hemis = heart_attack['Hemisphere'].values.reshape(-1, 1)
y_meduse = heart_attack['Medication Use'].values.reshape(-1, 1)
y_stresslvl = heart_attack['Stress Level'].values.reshape(-1, 1)
y_sedhrs = heart_attack['Sedentary Hours Per Day'].values.reshape(-1, 1)
y_income = heart_attack['Income'].values.reshape(-1, 1)
y_bmi = heart_attack['BMI'].values.reshape(-1, 1)
y_trig = heart_attack['Triglycerides'].values.reshape(-1, 1)
y_phydays = heart_attack['Physical Activity Days Per Week'].values.reshape(-1, 1)
y_sleephrs = heart_attack['Sleep Hours Per Day'].values.reshape(-1, 1)
y_country = heart_attack['Country'].values.reshape(-1, 1)

# Normalize data using Standard Scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Add intercept term to features
X_scaled = np.hstack((np.ones((len(y), 1)), X_scaled))
y_linear_pred# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

# Initialize linear regression model
linear_model = LinearRegression()

# Fit the model
linear_model.fit(X_linear, y_linear)
# Get predictions
y_linear_pred = linear_model.predict(X_linear)

# Calculate metrics for linear regression
linear_mse = mean_squared_error(y_linear, y_linear_pred)
r2_score = linear_model.score(X_linear, y_linear)


print('Linear Regression Metrics:')
print(f'Mean Squared Error (MSE): {linear_mse:.2f}')
print(f'R-squared (R2): {r2_score:.2f}')


NameError: name 'y' is not defined

time: 16 ms (started: 2023-12-13 20:03:55 -05:00)
